# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [5]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [6]:
# Run this code:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [26]:
first = salaries.head()
print(first)
salaries.describe()

                  Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4   

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,25161.000000,8022.000000
mean,34.507604,86786.999790,32.788558
std,9.252077,21041.354602,12.112573
min,10.000000,7200.000000,2.650000
25%,20.000000,76266.000000,21.200000
50%,40.000000,90024.000000,35.600000
75%,40.000000,96060.000000,40.200000
max,40.000000,300000.000000,109.000000


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
df_salaries = salaries.isna().sum()
print(df_salaries)

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [18]:
hourly=salaries['Salary or Hourly'].value_counts()['Hourly']
salary=salaries['Salary or Hourly'].value_counts()['Salary']
print('Hourly '+str(hourly))
print('Salary '+str(salary))

Hourly 8022
Salary 25161


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [25]:
dept_emp_num =  salaries.groupby('Department')['Department'].count()
print(dept_emp_num )

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [16]:
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy import stats
# Since we have an N of 8022 and only one sample (Hourly workers), we go for a Z-test
# H0: Hourly rate == 30
# H1: Hourly rate != 30
salaries_hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']
salaries_hourly['Hourly Rate']

pop_mean = np.mean(salaries_hourly['Hourly Rate'])

z_score, p_value = ztest(salaries_hourly['Hourly Rate'],value=30)
alpha = 0.05
critical_value = stats.norm.ppf(1 - alpha/2)
reject_null = np.abs(z_score) > critical_value

# Output the results
print(f"Sample Mean: {np.mean(salaries_hourly['Hourly Rate'])}")
print(f"Population Mean (Hypothesized): {pop_mean}")
print(f"Z-score: {z_score}")
print(f"P-value: {p_value}")
print(f"Critical value: {critical_value}")
print(f"Reject the null hypothesis: {reject_null}")


Sample Mean: 32.78855771628024
Population Mean (Hypothesized): 32.78855771628024
Z-score: 20.6198057854942
P-value: 1.8228873859286195e-94
Critical value: 1.959963984540054
Reject the null hypothesis: True


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [10]:
# Subset of the dataset: only Police department
salaries_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

In [18]:
# Since we have an N of 13404 and only one sample (salary of the police), we go for a Z-test
# H0: Annual Salary == 86000
# H1: Annual Salary > 86000

print(salaries_police['Annual Salary'])

pop_mean = np.mean(salaries_police['Annual Salary'])

z_score, p_value = ztest(salaries_police['Annual Salary'],value=86000)
alpha = 0.05
critical_value = stats.norm.ppf(1 - alpha/2)
reject_null = np.abs(z_score) > critical_value


# Output the results
print(f"Sample Mean: {np.mean(salaries_police['Annual Salary'])}")
print(f"Population Mean (Hypothesized): {86000}")
print(f"Z-score: {z_score}")
print(f"P-value: {p_value}")
print(f"Critical value: {critical_value}")
print(f"Reject the null hypothesis: {reject_null} if true means salary of police officers is greater than 86000")



# your code here

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13404, dtype: float64
Sample Mean: 86486.41450313339
Population Mean (Hypothesized): 86000
Z-score: 3.081997005712994
P-value: 0.002056169057509571
Critical value: 1.959963984540054
Reject the null hypothesis: True if true means salary of police officers is greater than 86000


Using the `crosstab` function, find the department that has the most hourly workers. 

In [19]:
# Filter the dataset to include only hourly workers
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

# Use crosstab to count the number of hourly workers in each department
crosstab_result = pd.crosstab(hourly_workers['Department'], hourly_workers['Salary or Hourly'])

# Find the department with the most hourly workers
most_hourly_workers_department = crosstab_result['Hourly'].idxmax()

most_hourly_workers_department, crosstab_result.loc[most_hourly_workers_department, 'Hourly']


('STREETS & SAN', 1862)

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [25]:
import pandas as pd
from scipy import stats

streets_and_san_hourly = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN')]

# Extract the hourly wages
hourly_wages = streets_and_san_hourly['Hourly Rate'].dropna()

# Perform a one-sample t-test
t_statistic, p_value = stats.ttest_1samp(hourly_wages, 35, alternative='less')

t_statistic, p_value

print(f"{t_statistic}--T stat-Negative means sample wage is lower than 35, {p_value} low p value allows to say this with confidence")

-9.567447887848152--T stat-Negative means sample wage is lower than 35, 1.6689265282353859e-21 low p value allows to say this with confidence
